In [2]:
import collections 
import d2lzh as d2l
from mxnet import gluon,init,nd
from mxnet.contrib import text
from mxnet.gluon import data as gdata,loss as gloss,nn,rnn,utils as gutils
import os 
import random
import tarfile



In [3]:
# 读取数据集

def download_imdb(data_dir = "../data/"):
    # url = ("http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz")
    # sha1 = "01ada507287d82875905620988597833ad4e0903"
    # fname = gutils.download(url, data_dir, sha1_hash=sha1)
    fname = "../data/aclImdb_v1.tar.gz"
    
    with tarfile.open(fname, 'r') as f:
        f.extractall(data_dir)
        
download_imdb()

In [4]:
# 读取训练数据集和测试数据集；每个样本是一条评论以及其对应的标签：1：正面；0：负面
def read_imdb(folder = 'train') :
    data = []
    for label in ['pos','neg']:
        folder_name = os.path.join("../data/aclImdb/",folder, label)
        for file in os.listdir(folder_name):
            with open(os.path.join(folder_name, file), 'rb') as f:
                review = f.read().decode("utf-8").replace("\n","").lower()
                data.append([review ,1 if label == 'pos' else 0])
    random.shuffle(data)
    return data
        
train_data,test_data = read_imdb("train"),read_imdb("test")

In [5]:
# 预处理数据

def get_tokenized_imdb(data): 
    def tokenizer(text):
        return [tok.lower() for tok in text.split(" ")]
    return [tokenizer(review) for review,_ in data]

# 创建词典
def get_vocab_imdb(data):
    tokenized_data = get_tokenized_imdb(data)
    counter = collections.Counter([tk for st in tokenized_data for tk in st])
    return text.vocab.Vocabulary(counter, min_freq = 5, reserved_tokens = ['<pad>'])

vocab = get_vocab_imdb(train_data)

'# words in vocab:', len(vocab)

('# words in vocab:', 46152)

In [6]:
# 由于每条评论长度不一，不以直接组合成小批量
# 通过词典转换成词索引，然后通过截断或补 '<pad>' (padding) 符号来将每条评论长度固定为500

def preprocess_imdb(data,vocab):
    max_l = 500 
    
    def pad(x):
        return x[:max_l] if len(x)>max_l else x + [vocab.token_to_idx['<pad>']] * (max_l - len(x))
    
    tokenized_data = get_tokenized_imdb(data)
    features = nd.array([pad(vocab.to_indices(x)) for x in tokenized_data])
    labels = nd.array([score for _,score in data])
    
    return features,labels

In [7]:
# 创建数据迭代器，每次迭代一个小批量的数据
batch_size = 64
train_set = gdata.ArrayDataset(*preprocess_imdb(train_data,vocab))
test_set = gdata.ArrayDataset(*preprocess_imdb(test_data, vocab))
train_iter = gdata.DataLoader(train_set, batch_size, shuffle = True)
test_iter = gdata.DataLoader(test_set, batch_size)


In [8]:
for X,y in train_iter:
    print("X",X.shape, "y",y.shape)
    break
    
"# batches:",len(train_iter)

X (64, 500) y (64,)


('# batches:', 391)

In [ ]:
# 使用循环